In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json
/kaggle/input/dm-2024-isa-5810-lab-2-homework/sampleSubmission.csv
/kaggle/input/dm-2024-isa-5810-lab-2-homework/data_identification.csv
/kaggle/input/dm-2024-isa-5810-lab-2-homework/emotion.csv


In [2]:
idtags = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/data_identification.csv')
idtags = idtags.sort_values(by='tweet_id').reset_index(drop=True)
idtags

,tweet_id,identification
0,0x1c7f0f,test
1,0x1c7f10,train
2,0x1c7f11,train
3,0x1c7f12,test
4,0x1c7f13,test
...,...,...
1867530,0x38fe19,train
1867531,0x38fe1a,train
1867532,0x38fe1b,test
1867533,0x38fe1c,train


In [3]:

tweet_data = pd.read_json('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json',lines=True)

_source_normalized = pd.json_normalize(tweet_data['_source'])

tweet_data = pd.concat([tweet_data.drop(columns=['_source']), _source_normalized], axis=1)

tweet_data = tweet_data.sort_values(by='tweet.tweet_id').reset_index(drop=True)

tweet_data


,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,62,hashtag_tweets,2017-05-14 11:39:43,tweets,[],0x1c7f0f,"@JZED74 While inappropriate AF, he likely wasn..."
1,242,hashtag_tweets,2015-05-16 10:36:47,tweets,[BlackMirror],0x1c7f10,o m g Shut Up And Dance though #BlackMirror <LH>
2,915,hashtag_tweets,2016-10-15 20:46:37,tweets,"[twitch, Destinybeta, Destiny, Destiny2, Desti...",0x1c7f11,On #twitch <LH> on the #Destinybeta #Destiny #...
3,756,hashtag_tweets,2016-02-14 15:55:45,tweets,[],0x1c7f12,I tried to figure out why you mean so much to ...
4,213,hashtag_tweets,2016-07-25 17:05:35,tweets,"[auspol, fizza]",0x1c7f13,"The only “big plan” you ever had in your life,..."
...,...,...,...,...,...,...,...
1867530,77,hashtag_tweets,2015-02-01 18:04:28,tweets,"[WesHoolahan, WALvIRL, COYBIG]",0x38fe19,Where is #WesHoolahan?! #WALvIRL #COYBIG <LH>
1867531,25,hashtag_tweets,2016-12-20 17:19:58,tweets,"[not, maga]",0x38fe1a,@mattmfm Fake news! <LH> propagated by Tumpkin...
1867532,611,hashtag_tweets,2016-02-14 03:24:55,tweets,[],0x38fe1b,I told myself I'd be twitter famous. twitter m...
1867533,639,hashtag_tweets,2016-09-13 06:31:27,tweets,[],0x38fe1c,..today was brutal ..#Hungover


In [4]:
tweet_data['tweet.text'] =tweet_data['tweet.text'].str.replace("@", '', regex=False)
tweet_data['tweet.text'] =tweet_data['tweet.text'].str.replace("#", '', regex=False)
tweet_data['tweet.text'] =tweet_data['tweet.text'].str.replace(">", '', regex=False)
tweet_data['tweet.text'] =tweet_data['tweet.text'].str.replace("LH", '', regex=False)
tweet_data['tweet.text'] =tweet_data['tweet.text'].str.replace("<", '', regex=False)
tweet_data['tweet.text'] =tweet_data['tweet.text'].str.replace(".", '', regex=False)
tweet_data['tweet.text'] =tweet_data['tweet.text'].str.replace(",", '', regex=False)



In [5]:
tweet_data['tweet.text']

0          JZED74 While inappropriate AF he likely wasn't...
1                o m g Shut Up And Dance though BlackMirror 
2          On twitch  on the Destinybeta Destiny Destiny2...
3          I tried to figure out why you mean so much to ...
4          The only “big plan” you ever had in your life ...
                                 ...                        
1867530              Where is WesHoolahan?!  WALvIRL COYBIG 
1867531    mattmfm Fake news!  propagated by Tumpkins  no...
1867532    I told myself I'd be twitter famous twitter ma...
1867533                           today was brutal  Hungover
1867534    Love it when I sun burn my forehead!! NOT!! 😫😱...
Name: tweet.text, Length: 1867535, dtype: object

In [6]:
mask=idtags['identification']=='train'
df_train=tweet_data[mask].reset_index(drop=True)
df_test=tweet_data[~mask].reset_index(drop=True)

In [7]:
df_train

,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,242,hashtag_tweets,2015-05-16 10:36:47,tweets,[BlackMirror],0x1c7f10,o m g Shut Up And Dance though BlackMirror
1,915,hashtag_tweets,2016-10-15 20:46:37,tweets,"[twitch, Destinybeta, Destiny, Destiny2, Desti...",0x1c7f11,On twitch on the Destinybeta Destiny Destiny2...
2,939,hashtag_tweets,2016-07-04 07:22:56,tweets,[],0x1c7f14,A nice sunny wak this morning not many aroud ...
3,181,hashtag_tweets,2016-04-16 12:53:40,tweets,"[Confession, NationalCandyCornDay, CouldEatThe...",0x1c7f15,I'm one of those people who love candy corn a ...
4,970,hashtag_tweets,2017-04-22 17:50:28,tweets,[],0x1c7f16,metmuseum What are these? They look like somet...
...,...,...,...,...,...,...,...
1455558,922,hashtag_tweets,2016-12-06 11:10:57,tweets,[],0x38fe18,LJPBR FifthHarmony Um My vote For FifthHarmon...
1455559,77,hashtag_tweets,2015-02-01 18:04:28,tweets,"[WesHoolahan, WALvIRL, COYBIG]",0x38fe19,Where is WesHoolahan?! WALvIRL COYBIG
1455560,25,hashtag_tweets,2016-12-20 17:19:58,tweets,"[not, maga]",0x38fe1a,mattmfm Fake news! propagated by Tumpkins no...
1455561,639,hashtag_tweets,2016-09-13 06:31:27,tweets,[],0x38fe1c,today was brutal Hungover


In [8]:
df_test

,_score,_index,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,62,hashtag_tweets,2017-05-14 11:39:43,tweets,[],0x1c7f0f,JZED74 While inappropriate AF he likely wasn't...
1,756,hashtag_tweets,2016-02-14 15:55:45,tweets,[],0x1c7f12,I tried to figure out why you mean so much to ...
2,213,hashtag_tweets,2016-07-25 17:05:35,tweets,"[auspol, fizza]",0x1c7f13,The only “big plan” you ever had in your life ...
3,603,hashtag_tweets,2017-01-21 19:25:33,tweets,[],0x1c7f17,Looking back on situations old & new recent or...
4,609,hashtag_tweets,2017-04-25 16:36:47,tweets,[],0x1c7f18,jasoninthehouse Why do you insist on talking a...
...,...,...,...,...,...,...,...
411967,497,hashtag_tweets,2016-12-05 19:57:34,tweets,"[traitor, treason]",0x38fe04,"""The Grand Bargain"" The Great American Betraya..."
411968,187,hashtag_tweets,2017-07-15 19:43:46,tweets,"[FosterCare, roadtoadoption, Adoption]",0x38fe06,I get to be a 1 year old girl's new mama start...
411969,139,hashtag_tweets,2017-07-13 07:01:50,tweets,[artworld],0x38fe13,Asian dude with dangly gold earrings is back &...
411970,251,hashtag_tweets,2016-08-22 22:13:06,tweets,"[nba, cantmakeaJ]",0x38fe14,I think kostakoufos might be the worst player ...


In [9]:
emotion_data = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/emotion.csv')
emotion_data = emotion_data.sort_values(by='tweet_id').reset_index(drop=True)
emotion_data

,tweet_id,emotion
0,0x1c7f10,joy
1,0x1c7f11,anticipation
2,0x1c7f14,joy
3,0x1c7f15,joy
4,0x1c7f16,disgust
...,...,...
1455558,0x38fe18,sadness
1455559,0x38fe19,anticipation
1455560,0x38fe1a,surprise
1455561,0x38fe1c,disgust


In [10]:
X_data=df_train['tweet.text']
y_data=emotion_data['emotion']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.20, random_state=42)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

# build analyzers (bag-of-words)
BOW_2000 = CountVectorizer(max_features=2000, tokenizer=nltk.word_tokenize, stop_words='english') 

# apply analyzer to training data
BOW_2000.fit(X_train)

train_data_BOW_features_2000 = BOW_2000.transform(X_train)

## check dimension
train_data_BOW_features_2000.shape

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1164450, 2000)

In [13]:
X_train = BOW_2000.transform(X_train)
X_val = BOW_2000.transform(X_val)


In [14]:
tokens=BOW_2000.get_feature_names_out()

token_counts = np.array(X_train.sum(axis=0)).flatten()

token_freq = sorted(zip(tokens, token_counts), key=lambda x: x[1], reverse=True)

for token, freq in token_freq[0:100]:
    print(f"{token}: {freq}")


!: 424220
?: 155176
's: 100856
n't: 87921
&: 77904
just: 67603
’: 65370
love: 65355
life: 63343
today: 54548
``: 50108
'': 49154
:: 48406
day: 47721
like: 46400
'm: 46213
god: 45408
good: 37278
people: 35167
time: 33928
got: 29608
-: 29235
): 28755
know: 27776
thank: 26290
make: 26274
2017: 26118
pips: 26080
u: 25849
new: 25238
s: 23579
great: 22327
realdonaldtrump: 21930
(: 21691
did: 21663
;: 21592
happy: 21430
're: 20543
work: 20380
ca: 20196
need: 20069
t: 19815
going: 19600
thanks: 19581
2: 19114
want: 19031
come: 18863
really: 18834
best: 18129
let: 17301
way: 17132
world: 16967
right: 16757
does: 16490
think: 15982
've: 15678
say: 15233
hope: 15085
feel: 14969
things: 14869
lots: 14746
man: 14728
total: 14715
week: 14049
closed: 13884
follow: 13769
night: 13634
morning: 13614
better: 13197
eurusd: 13039
m: 12548
1: 12499
amazing: 12282
feeling: 11982
year: 11956
tonight: 11893
10: 11616
days: 11558
help: 11357
play: 11317
na: 11138
3: 10979
': 10855
moments: 10825
believe: 10758

In [15]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_num = label_encoder.fit_transform(y_train)  
y_val_num = label_encoder.fit_transform(y_val)


In [16]:
print('X_train.shape: ', X_train.shape)
print('y_train_num.shape: ', y_train_num.shape)
print('X_val.shape: ', X_val.shape)
print('y_val_num.shape: ', y_val_num.shape)

X_train.shape:  (1164450, 2000)
y_train_num.shape:  (1164450,)
X_val.shape:  (291113, 2000)
y_val_num.shape:  (291113,)


In [17]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train_num)
dval = xgb.DMatrix(X_val, label=y_val_num)

In [18]:
params = {
    'objective': 'multi:softmax',  
    'eval_metric': 'merror', 
    'num_class': len(label_encoder.classes_), 
    'max_depth': 30,
    'learning_rate': 0.5,
}

bst = xgb.train(params, dtrain, num_boost_round=300, evals=[(dtrain, 'train') , (dval, 'valid')],verbose_eval=10)


[0]	train-merror:0.51946	valid-merror:0.52970
[10]	train-merror:0.47741	valid-merror:0.50903
[20]	train-merror:0.45751	valid-merror:0.50207
[30]	train-merror:0.44346	valid-merror:0.49869
[40]	train-merror:0.43189	valid-merror:0.49588
[50]	train-merror:0.42187	valid-merror:0.49474
[60]	train-merror:0.41250	valid-merror:0.49344
[70]	train-merror:0.40450	valid-merror:0.49316
[80]	train-merror:0.39707	valid-merror:0.49248
[90]	train-merror:0.38983	valid-merror:0.49197
[100]	train-merror:0.38316	valid-merror:0.49126
[110]	train-merror:0.37696	valid-merror:0.49118
[120]	train-merror:0.37140	valid-merror:0.49115
[130]	train-merror:0.36620	valid-merror:0.49087
[140]	train-merror:0.36136	valid-merror:0.49054
[150]	train-merror:0.35665	valid-merror:0.49050
[160]	train-merror:0.35268	valid-merror:0.49060
[170]	train-merror:0.34843	valid-merror:0.49054
[180]	train-merror:0.34484	valid-merror:0.49046
[190]	train-merror:0.34164	valid-merror:0.49053
[200]	train-merror:0.33829	valid-merror:0.49062
[21

In [19]:
y_pred_num = bst.predict(dval)  

y_pred_str = label_encoder.inverse_transform(y_pred_num.astype(int))


In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_val, y_pred=y_pred_str))

              precision    recall  f1-score   support

       anger       0.46      0.16      0.23      8062
anticipation       0.61      0.48      0.54     50129
     disgust       0.40      0.29      0.33     27789
        fear       0.64      0.30      0.41     12679
         joy       0.50      0.81      0.62    102986
     sadness       0.46      0.40      0.43     38793
    surprise       0.54      0.17      0.25      9790
       trust       0.51      0.26      0.35     40885

    accuracy                           0.51    291113
   macro avg       0.51      0.36      0.40    291113
weighted avg       0.51      0.51      0.48    291113



In [21]:
X_test=df_test['tweet.text']
X_test=BOW_2000.transform(X_test)

dtest=xgb.DMatrix(X_test)

In [22]:
prediction_num=bst.predict(dtest)
prediction_str= label_encoder.inverse_transform(prediction_num.astype(int))


In [23]:
prediction=pd.DataFrame(data=prediction_str,columns=["emotion"])
prediction

,emotion
0,joy
1,sadness
2,anticipation
3,anticipation
4,disgust
...,...
411967,sadness
411968,trust
411969,joy
411970,sadness


In [24]:
df_result=pd.concat([df_test['tweet.tweet_id'], prediction['emotion']], axis=1)
df_result=df_result.rename(columns={"tweet.tweet_id":"id"})
df_result

,id,emotion
0,0x1c7f0f,joy
1,0x1c7f12,sadness
2,0x1c7f13,anticipation
3,0x1c7f17,anticipation
4,0x1c7f18,disgust
...,...,...
411967,0x38fe04,sadness
411968,0x38fe06,trust
411969,0x38fe13,joy
411970,0x38fe14,sadness


In [25]:
df_result.to_csv("emotion_prediction.csv",index = False)